# Instagram Like Prediction @310ai Competition - Modeling, Visualization & Evaluation

This notebook is a continuation of ***Data*** notebook. In this part of project we will train the predictive model, analyze it, visualize its output and evaluate it. The reason for this separation, is to make the notebooks easier to understand and less dependent on each other, especially for this case since the Instagram showed that it reguralry changing its api and bot mitigation methods. Multiple stages of this project have checkpoints, thus the notebook will run without issues.

As we pointed out in the previous notebook, we will use **XGBoost 1.7** as the algorithm. So let's without further ado, dive into it.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

First thing first, we must load the data, also we have some identifying features, called shortcode, username and id, which will not affect the accuracy of the model, thus we can drop these features all together.

In [2]:
df = pd.read_csv('Data/main_dataset.csv')
df.drop(columns=['Unnamed: 0', 'shortcode', 'username', 'id'], inplace=True)
df.head()

,post_type,like,comment,object_1,object_2,object_3,object_4,object_5,object_6,category_name,...,reel_count,reel_avg_view,reel_avg_comment,reel_avg_like,reel_avg_duration,reel_frequency,media_count,media_avg_comment,media_avg_like,media_frequency
0,GraphSidecar,363269,6844,No Object,No Object,No Object,No Object,No Object,No Object,Digital creator,...,1256,9406907.25,13544.50000,6.345007e+05,92.466333,18.554887,7393,8635.166667,410027.250,1.090777
1,GraphSidecar,546578,11354,No Object,No Object,No Object,No Object,No Object,No Object,Digital creator,...,1256,9406907.25,13544.50000,6.345007e+05,92.466333,18.554887,7393,8635.166667,410027.250,1.090777
2,GraphImage,734124,11937,No Object,No Object,No Object,No Object,No Object,No Object,Digital creator,...,1256,9406907.25,13544.50000,6.345007e+05,92.466333,18.554887,7393,8635.166667,410027.250,1.090777
3,GraphSidecar,7720568,42249,2 people,people playing football,people playing soccer,ball,No Object,No Object,Athlete,...,387,18249237.58,28318.83333,3.607850e+06,32.616333,17.036130,3475,52543.166670,7980593.083,2.203446
4,GraphImage,14451079,103278,1 person,baby,No Object,No Object,No Object,No Object,Athlete,...,387,18249237.58,28318.83333,3.607850e+06,32.616333,17.036130,3475,52543.166670,7980593.083,2.203446


Now we can separate independent and dependent variables. Usually we cal independent variables X and dependent variables y.

In [3]:
X, y = df.loc[:, df.columns != 'like'].copy(), df[['like']].copy()

As we said in the previous notebook, XGBoost since the version of 1.7 is capable of working with categorical variables, but before casting the categorical variables through the model, we must change the type of those features to `categorical` in pandas.

In [4]:
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

Now we can split the data into training and validation. We will use the 25% of data as validation set.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=69)

XGBoost has its own class for storing datasets, it's called `DMatrix`. This class is highly optimized for memory and speed. Converting to this datatype is neccessary for passing values to XGBoost. Also, in this cell we will define hyperparameters value for model. Have in mind that since we will perform a cross-validation, parameters are actually list of parameters to find the best one among their combination.

In [26]:
model = xgb.XGBRegressor(tree_method='hist', enable_categorical=True)

In [31]:
params = {
    'booster': ['gbtree', 'dart'],
    'eta': np.arange(0.25, 1.1, 0.25),
    'subsample': np.arange(0.25, 1.1, 0.25),
    'refresh_leaf': [0, 1]
}
grid = GridSearchCV(model, param_grid=params, return_train_score=True, cv=5, verbose=1)
grid.fit(X, y)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, ma...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'booster': ['gbtree', 'dart'],
                         'eta': array([0.25, 0.5 , 0.75, 1.  ]),
                         'refresh_leaf': [0, 1],
                         'subsample': array([0.25, 0.5 , 0.75, 1.  ])},
             return_train_score=True, verbose=1)

In [37]:
results = pd.DataFrame(grid.cv_results_)
results.to_csv('results.csv')